# Quantum Computation - Week 8

&nbsp;

## Contents

&nbsp;

1. [Quantum Counting](#qcount)

    1.1 [Implementation](#impl)

&nbsp;

# 1. Quantum Counting <a id='qcount'></a>

&nbsp;

Whereas Grover’s algorithm attempts to find a solution to the Oracle, the quantum counting algorithm aims to tells us how many of these solutions there are. This algorithm is interesting as it combines both quantum search and quantum phase estimation.

&nbsp;

## The problem

&nbsp;

Consider a finite set $\{0,1\}^n$ of size $N=2^n$ and a set $B$ of "solutions" (that is a subset of $ \{0,1\}^n $). Define:

$ \begin{cases} f : \left\{0,1\right\}^n \to \{0,1\} \\ f(x) = \begin{cases} 1  & x \in B \\ 0  &  x \notin B \end{cases} \end{cases}$

In other words, $f$ is the indicator function of $B$.

Calculate the number of solutions $ M = \left\vert f^{-1}(1) \right\vert = \vert B \vert$

## Introduction

&nbsp;

In quantum counting, we simply use the quantum phase estimation algorithm to find an eigenvalue of a Grover search iteration. 

Remember that an iteration of Grover’s algorithm can be visualized as vector rotations on the $\mathbb{C}^N$ plane spanned by two statevectors: $| s \rangle$, a uniform superposition of all input states

$$| s \rangle = \frac{1}{\sqrt{N}} \sum_{x = 0}^{N -1} | x
\rangle $$

and a *marked* state $|w\rangle$, also known as the "winner".

<img src="img\grover_search.png" width="400 px" align="center">

These statevectors are not perpendicular as $| w \rangle$ occurs in the superposition $|s\rangle$ with amplitude $N^{-1/2}$ as well. We can, however, introduce an additional state $|s'\rangle$ that is in the span of these two vectors, which is perpendicular to $| w \rangle$ and is obtained from $|s \rangle$ by removing $| w \rangle$ and rescaling. 



An iteration of Grover's algorithm $G$, rotates the state vector by $\theta$ in the $|\omega\rangle$, $|s’\rangle$ basis:

<img src="img\quantum_counting1.svg" width="300 px" align="center">


The percentage number of solutions in our search space affects the difference between $|s\rangle$ and $|s’\rangle$. For example, if there are not many solutions, $|s\rangle$ will be very close to $|s’\rangle$ and $\theta$ will be very small. It turns out that the eigenvalues of the Grover iterator are $e^{\pm i\theta}$, and we can extract this using quantum phase estimation (QPE) to estimate the number of solutions ($M$).

## Grover's iteration as an operator

&nbsp;

In the $|\omega\rangle$,$|s’\rangle$ basis we can write the Grover iterator as the matrix:

$$
G =
\begin{pmatrix}
\cos{\theta} && -\sin{\theta}\\
\sin{\theta} && \cos{\theta}
\end{pmatrix}
$$

The matrix $G$ has eigenvectors:

$$
\begin{pmatrix}
-i\\
1
\end{pmatrix}
,
\begin{pmatrix}
i\\
1
\end{pmatrix}
$$

With the aforementioned eigenvalues $e^{\pm i\theta}$. Fortunately, we do not need to prepare our register in either of these states, the state $|s\rangle$ is in the space spanned by $|\omega\rangle$, $|s’\rangle$, and thus is a superposition of the two vectors.
$$
|s\rangle = \alpha |\omega\rangle + \beta|s'\rangle
$$

As a result, the output of the QPE algorithm will be a superposition of the two phases, and when we measure the register we will obtain one of these two values! We can then use some simple maths to get our estimate of $M$.

<img src="img\quantum_counting2.svg" width="600 px" align="center">

Recall, from the previous class, that measured bit values $\phi_1, \cdots, \phi_n$ give an estimate of the phase $\phi \approx 0.\phi_1 \cdots \phi_n$, or $\phi \approx \frac{\phi_1}{2}+ \cdots + \frac{\phi_n}{2^n}$.


## 1.1 Implementation <a id='impl'></a>

We choose to ‘count’ on the first 4 qubits (we call the number of counting qubits $t$, so $t = 4$), and to 'search' through the last 4 qubits ($n = 4$). Since we only care about the measurement of the counting qubits, we have initialised 4 classical bits to measure to.
With that done, we can start creating the building blocks of our circuit.

**Module Imports**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

# importing Qiskit
import qiskit
from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

from qiskit.aqua.circuits.gates.multi_control_toffoli_gate import _cccx

# import basic plot tools
from qiskit.visualization import plot_histogram

In [ ]:
# Plot results
def show_results(D):
    # D is a dictionary with classical bits as keys and count as value
    # example: D = {'000': 497, '001': 527}
    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    plt.show()

# Execute circuit, display a histogram of the results
def execute_locally(qc, draw_circuit=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    
    # Print the results
    print("simulation: \n\n", result_counts)
    show_results(result_counts)

### Controlled-Grover Iteration

We have already covered Grover iterations in the Grover’s algorithm section. Here is an example with an Oracle we know has 5 solutions ($M = 5$) of 16 states ($N = 2^n = 16$), combined with a diffusion operator:

In [ ]:
def example_grover_iteration():
    """Small circuit with 5/16 solutions"""
    # Do circuit
    q = QuantumRegister(4, 'q')
    circ = QuantumCircuit(q)
    # Oracle
    circ.h(3)
    _cccx(circ,q)
    circ.x(0)
    _cccx(circ,q)
    circ.x(0)
    circ.x(1)
    _cccx(circ,q)
    circ.x(1)
    circ.x(2)
    _cccx(circ,q)
    circ.x(2)
    circ.x(1)
    circ.x(2)
    _cccx(circ,q)
    circ.x(2)
    circ.x(1)
    circ.h(3)
    # Diffusion Operator
    circ.z(3)
    for qubit in q[:3]:
        circ.h(qubit)
        circ.x(qubit)
    _cccx(circ,q)
    for qubit in q[:3]:
        circ.x(qubit)
        circ.h(qubit)
    circ.z(3)
    return circ

<div class="alert alert-block alert-info">
    
**Quantum circuits as (controlled) multi-qubit gates**

&nbsp;

Notice the python function takes no input and returns a `QuantumCircuit` object with 4 qubits. In the past the functions you created might have modified an existing circuit, but a function like this allows us to turn the `QuantumCircuit` object into a single gate we can then control.

We can use `.to_gate()` and `.control()` to create a controlled gate from a circuit.

These custom gates can then be added to the circuit using the `.apend(gate, qargs = [qubits])` method on the `QuantumCircuit` object.

</div>


We will call our Grover iterator `grit` and the controlled Grover iterator `cgrit`

In [ ]:
# Create controlled-Grover
grit = example_grover_iteration().to_gate()
cgrit = grit.control(1)

In [ ]:
qr = QuantumRegister(4, 'q')
qc = QuantumCircuit(qr)
qc.append(grit, qargs=qr)
qc.draw(output='mpl')

In [ ]:
qr = QuantumRegister(5, 'q')
qc = QuantumCircuit(qr)
qc.append(cgrit, qargs= [qr[4]] + qr[:4])
qc.draw(output='mpl')

### Inverse QFT <a id='inv_qft'></a>
We now need to create an inverse QFT. This code implements the QFT on n qubits:

In [ ]:
def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cu1(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

<div class="alert alert-block alert-info">
    
**Inverting quantum circuits as gates**

&nbsp;

A circuit converted into a `Gate` object can be inverted by adding the `.inverse()` method.

</div>


In [ ]:
iqft = qft(QuantumCircuit(4),4).to_gate().inverse()

### Putting it Together 

&nbsp;

<div class="alert alert-block alert-warning">
    
**Exercise 1**

&nbsp;

<img src="img\quantum_counting2.svg" width="600 px" align="center">


1. Apply the quantum counting algorithm represented above, for `t=4` counting qubits, and `n=4` *searching* bits. What are the results after simulation?

</div>



In [ ]:
# Check circuit
qc.draw(output='mpl')

In [ ]:
# Simulate circuit
simulator = Aer.get_backend('qasm_simulator')
job = execute(circ, simulator, shots=2048 )
hist = job.result().get_counts()
plot_histogram(hist)

We can see two values stand out, having a much higher probability of measurement than the rest. These two values correspond to $e^{i\theta}$ and $e^{-i\theta}$, but we can’t see the number of solutions yet. We need to little more processing to get this information, so first let us get our output into something we can work with (an `int`).

Get the string of the most probable result from our output data:

In [ ]:
measured_str = max(hist, key=hist.get)

Store this as an integer:

In [ ]:
measured_int = int(measured_str,2)
print("Register Output = %i" % measured_int)

### Finding the Number of Solutions (M)

<div class="alert alert-block alert-warning">
    
**Exercise 2**

&nbsp;

1. Create a function, `calculate_M(integ, t, n)` that takes as input the decimal integer output of our register, the number of counting qubits ($t$) and the number of searching qubits ($n$); and returns the numer of solutions `M`.

</div>

**Tips:**

* First we want to get $\theta$ from `measured_int`. Remember that QPE gives us a measured $\text{value} = 2^n \phi$ from the eigenvalue $e^{2\pi i\phi}$, so to get $\theta$ we need to do:

$$ \theta = value \times \frac{2 \pi}{2^t} $$


* We can get the angle $\theta/2$ can from the inner product of $|s\rangle$ and $|s’\rangle$:

$$
\langle s'|s\rangle = \cos{\tfrac{\theta}{2}}
$$

And that the inner product of these vectors is:

$$
\langle s'|s\rangle = \sqrt{\frac{N-M}{N}}
$$

We can combine these equations, then use some trigonometry and algebra to show:

$$
N\sin^2{\frac{\theta}{2}} = M
$$

In [ ]:
def calculate_M(integ, t, n):
    


<div class="alert alert-block alert-warning">
    
**Exercise 3**

1.	Can you adapt the circuit to use more or less counting qubits to get a different precision in your result?

</div>